# Extract Workout Trainer Reviews and Stars from Play Store
This code uses Selenium to automatically load reviews from the Google Play store, then imports the HTML into beautiful soup to extract the review text and the star ratings

In [1]:
import sys
import re
import numpy as np
import pandas as pd
from pprint import pprint
import requests
from bs4 import BeautifulSoup

In [3]:
#Open up a google play review page
#and continuously scroll down to the bottom
#automatically click the "see more" button at the bottom then continue scrolling down
#Once a predefined number of button clicks have been make, copy all of the loaded html data
#for use in BeautifulSoup to extract reviews and star ratings
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome('/Users/jnaviaux/anaconda3/envs/Ongo_Science_Project/20190616-production/Ongo_wk3_code/chromedriver')


browser.get("https://play.google.com/store/apps/details?id=com.skimble.workouts&hl=en_US&showAllReviews=true")
time.sleep(1)

#Select main page and set up loop to continuously page down
main_page = browser.find_element_by_tag_name('body')


# Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
total_button_clicks = 70
while(total_button_clicks):
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        try: #if at the end of the page, try to click on the button to show more reviews
            button = browser.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div[2]/div/span/span')
            button.click()
            total_button_clicks-=1
            print('clicks left', total_button_clicks)
        except:
            print('couldnt find button')
            total_button_clicks-=1
            print('continuing with clicks left = ',total_button_clicks)
            continue
            
            
        finally:
            page_source_test = browser.page_source


clicks left 69
clicks left 68
clicks left 67
clicks left 66
clicks left 65
clicks left 64
clicks left 63
clicks left 62
clicks left 61
clicks left 60
clicks left 59
clicks left 58
clicks left 57
clicks left 56
clicks left 55
clicks left 54
clicks left 53
clicks left 52
clicks left 51
clicks left 50
clicks left 49
clicks left 48
clicks left 47
clicks left 46
clicks left 45
clicks left 44
clicks left 43
clicks left 42
clicks left 41
clicks left 40
clicks left 39
clicks left 38
clicks left 37
clicks left 36
clicks left 35
clicks left 34
clicks left 33
clicks left 32
clicks left 31
clicks left 30
clicks left 29
clicks left 28
clicks left 27
clicks left 26
clicks left 25
clicks left 24
clicks left 23
clicks left 22
clicks left 21
clicks left 20
clicks left 19
clicks left 18
clicks left 17
clicks left 16
clicks left 15
clicks left 14
clicks left 13
clicks left 12
clicks left 11
clicks left 10
clicks left 9
clicks left 8
clicks left 7
clicks left 6
clicks left 5
clicks left 4
clicks left 3
cl

In [4]:
from bs4 import BeautifulSoup

In [5]:
test_soup = BeautifulSoup(page_source_test, 'html5lib')

## Pull out reviews
Combine short and long reviews into single list

In [6]:
review_body = test_soup.find(class_='W4P4ne')
all_reviews = review_body.find_all(jsname='bN97Pc')
all_reviews_long_cutoff = [review.get_text() for review in all_reviews]

long_reviews = review_body.find_all(jsname='fbQN7e')
long_reviews_only = [review.get_text() for review in long_reviews]

In [7]:
#combine the list of long reviews with the shorter one to make a complete list of all reviews

#find indexes of blank reviews in "long reviews"
idxs = [i for i,x in enumerate(long_reviews_only) if not x]

for idx in idxs:
    long_reviews_only[idx] = all_reviews_long_cutoff[idx]


## Pull out star ratings

In [8]:
#try to get star rating out of each review
star_rating = review_body.find_all(class_='pf5lIe')

#pull out the rating given by every review
star_list = [int(i.find({'div':'aria-label'}).attrs['aria-label'][6]) for i in star_rating]

In [9]:
len(star_list)

11280

In [10]:
len(long_reviews_only)

11280

In [11]:
long_reviews_only[0:100]

['Wide range of workouts, so handy to browse and save for later, much rather be browsing workouts for the gym than social media, really easy to use, helpful images and descriptions of workouts especially for beginners!',
 "every device I've owned since galaxy s3- my s10e now has worked excellent! even on tablets I love these workouts they're very convenient and I've lost a lot of weight and feel great!",
 'Has various good workout. I have yet to get tired of doing these exercises. I recommend this app for everyone to try out.',
 'i love it, except it limits you a lot and monthly payment is just exaggerated',
 'The free version provides plenty of routines and flexibility in terms of type of workout and time duration',
 'Great app. it keeps getting better.',
 'Love the app. Keeps me working really hard',
 "I really enjoy this app. It was recommended to me by a personal trainer and I've been using it since. For some reason, this is the first workout program I have actually been sticking t

In [12]:
len(long_reviews_only)-star_list.count(5)

3846

## Combine reviews and stars into dataframe

In [13]:
workout_trainer_dict={'reviews':long_reviews_only,'stars':star_list}

In [14]:
import pandas as pd
workout_trainer_df = pd.DataFrame(data=workout_trainer_dict)

In [15]:
workout_trainer_df.head()

,reviews,stars
0,"Wide range of workouts, so handy to browse and...",5
1,every device I've owned since galaxy s3- my s1...,5
2,Has various good workout. I have yet to get ti...,4
3,"i love it, except it limits you a lot and mont...",4
4,The free version provides plenty of routines a...,5


In [16]:
import pickle

In [17]:
#save the imported reviews and star ratings to pickle so they won't be lost

In [18]:
workout_trainer_df.to_pickle("./workout_trainer_reviews_df.pkl")